In [11]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

In [26]:
pd.set_option('max_columns', None)

In [18]:
con = sqlite3.connect('database.sqlite')

In [19]:
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('sqlite_sequence',), ('Player_Attributes',), ('Player',), ('Match',), ('League',), ('Country',), ('Team',), ('Team_Attributes',)]


1. Which team scored the most points when playing at home?  

In [37]:
query = '''
SELECT home_team_api_id, team_long_name, MAX(home_team_goal) FROM Match
JOIN Team ON Match.home_team_api_id = Team.team_api_id;
'''

pd.read_sql_query(query, cnx)

,home_team_api_id,team_long_name,MAX(home_team_goal)
0,8640,PSV,10


2. Did this team also score the most points when playing away? 

In [39]:
query = '''
SELECT away_team_api_id, team_long_name, MAX(away_team_goal) FROM Match
JOIN Team ON Match.away_team_api_id = Team.team_api_id;
'''

pd.read_sql_query(query, cnx)

,away_team_api_id,team_long_name,MAX(away_team_goal)
0,9847,Paris Saint-Germain,9


3. How many matches resulted in a tie?

In [42]:
query = '''
SELECT Count(id) FROM Match
WHERE home_team_goal = away_team_goal
'''

pd.read_sql_query(query, cnx)

,Count(id)
0,6596


4. How many players have Smith for their last name? How many have 'smith' anywhere in their name?

In [126]:
query = '''
SELECT player_name FROM Player
'''

names = pd.read_sql_query(query, cnx)

In [127]:
names['player_name'] = names['player_name'].apply(lambda x: x.lower())

In [128]:
names['count'] = names['player_name'].apply(lambda x: 1 if 'smith' in x else 0)

In [129]:
names['count'].value_counts()

0    11042
1       18
Name: count, dtype: int64

In [130]:
names['last name'] = names['player_name'].apply(lambda x: x.split()[-1])

In [131]:
names['last name count'] = names['last name'].apply(lambda x: 1 if 'smith' in x else 0)

In [132]:
names['last name count'].value_counts()

0    11042
1       18
Name: last name count, dtype: int64

5. What was the median tie score? Use the value determined in the previous question for the number of tie games. *Hint:* PostgreSQL does not have a median function. Instead, think about the steps required to calculate a median and use the [`WITH`](https://www.postgresql.org/docs/8.4/static/queries-with.html) command to store stepwise results as a table and then operate on these results. 

In [143]:
# From a previous step we know that there were 8,640 ties. If we get all ties and sort by score, the 4,320th record should be our median.

query = '''
SELECT * FROM Match
WHERE home_team_goal = away_team_goal
ORDER BY home_team_goal ASC
'''

df = pd.read_sql_query(query, cnx)
df['home_team_goal'][4320]

1

In [141]:
# Alternatively, we can just call median on the column.
df['home_team_goal'].median()

1.0

6. What percentage of players prefer their left or right foot? *Hint:* Calculate either the right or left foot, whichever is easier based on how you setup the problem.

In [145]:
query = '''
SELECT preferred_foot FROM Player_Attributes
'''

foot = pd.read_sql_query(query, cnx)

In [150]:
foot['preferred_foot'].value_counts()

right    138409
left      44733
Name: preferred_foot, dtype: int64

In [152]:
len(foot['preferred_foot'])

183978

In [154]:
# 75 percent of players are right-footed
138409/183978

0.7523127765276283